In [ ]:
import numpy as np
import pandas as pd
import pathlib
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Manually downloaded data set from link:
# https://data.cityofnewyork.us/City-Government/Value-of-Energy-Cost-Savings-Program-Savings-for-B/bug8-9f3g
# (under "Export" option, retrieve CSV file)

# load data into pandas
fileName = "Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses.csv"
dataPath = pathlib.Path().resolve().joinpath("data", fileName)
# print(f"Path to data set: {dataPath}")
df = pd.read_csv(dataPath)

In [ ]:
df.head()

In [ ]:
print(f"df.shape: {df.shape}")
print(f"df.columns: {df.columns}")

In [ ]:
# 1 How many different companies are represented in the data set?
print("### 1 ###")
print(f"Number of unique companies: {df['Company Name'].str.lower().nunique()}\n")

# 2 What is the total number of jobs created for businesses in Queens?
print("### 2 ###")
print(f"Unique boroughs: {df['Borough'].unique()}")
print(f"Number of jobs created for businesses in Queens: {df.loc[df['Borough'] == 'QUEENS', :]['Job created'].sum()}\n")

# 3 How many different unique email domains names are there in the data set?
print("### 3 ###")
# get desired column 'company email' as a pandas series, convert strings to lowercase
# use apply function to get email domain for each element
# get unique email domains, compute sum

def get_email_domain(str_email):
    """Get domain from email address.
       Email address must have the at sign @,
       if not "None" is returned.
    """
    if str_email:
        try:
            regx_pattern = r'@[\w.]+'
            domain = re.search(regx_pattern, str_email).group()
        except AttributeError:
            domain = None
    else:
        domain = None

    return domain

# Add column with the email domain
df['email domain'] = df.loc[df['company email'].notnull(), 'company email'].str.lower().apply(lambda x : get_email_domain(x))

# Compute number of unique email domains
print(f"Number of unique email domains: {df['email domain'].nunique()}\n")

# 4 Considering only NTAs with at least 5 listed businesses, 
# what is the average total savings and the total jobs created for each NTA?
print("### 4 ###")
print(f"Compute the average total savings and the total jobs created for each NTA with at least 5 listed businesses...\n")
NTA_groups = df.groupby('Neighborhood Tabulation Area (NTA) (2020)')
results = NTA_groups.agg({'Total Savings': [('average total savings', np.mean)], 'Job created': [('total job created', np.sum)]})
results.columns = results.columns.get_level_values(1)
results = results.loc[NTA_groups.size() >= 5, :]

# 5 Save your result for the previous question as a CSV file.
print("### 5 ###")
file_name = 'results_stats_NTA_groups.csv'
results.to_csv(file_name, encoding='utf-8', index=True)
print(f"Results saved in {file_name}")

In [ ]:
results.head()

In [ ]:
# Create scatter plot of jobs created versus average savings. 
# Use both a standard and a logarithmic scale for the average savings.

# histogram on linear scale
sns.scatterplot(x='average total savings', y='total job created', data=results, s=150)
plt.xlabel('Average total savings')
plt.ylabel('Jobs created')
plt.title('jobs created vs average savings')
plt.show()

# histogram on log scale.
sns.scatterplot(x='average total savings', y='total job created', data=results, s=150)
plt.xscale('log')
plt.xlabel('Average total savings')
plt.ylabel('Jobs created')
plt.title('jobs created vs average savings')
plt.show()

# Create a histogram of the log of the average total savings.
# sns.histplot(np.log(results['Total Savings']), stat='probability', kde=True)
sns.histplot(np.log(results['average total savings']), stat='probability')
plt.xlabel('ln(Average total savings)')
plt.ylabel('Fraction of observations')
# plt.title('ln average total savings distribution plot')
plt.show()